In [5]:
import os
import mysql.connector
from mysql.connector import Error
import pandas as pd
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def get_db_config():
    """Get database configuration from environment variables."""
    return {
        'host': os.getenv('DB_HOST', 'localhost'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASSWORD'),
        'database': os.getenv('DB_NAME'),
        'port': int(os.getenv('DB_PORT', '3306')),
        'charset': os.getenv('DB_CHARSET', 'utf8mb4'),
        'collation': os.getenv('DB_COLLATION', 'utf8mb4_general_ci')
    }

def create_connection():
    """Create a database connection from .env configuration."""
    connection = None
    try:
        db_config = get_db_config()
        connection = mysql.connector.connect(**db_config)
        if connection.is_connected():
            print(f"Successfully connected to MariaDB at {db_config['host']}")
            print(f"Connected to server version {connection.get_server_info()}")
            print(f"Database: {db_config['database']}")
            return connection
    except Error as e:
        print(f"Error connecting to MariaDB: {e}")
        print("\nConnection details (excluding password):")
        safe_config = {k: v for k, v in get_db_config().items() if k != 'password'}
        print(safe_config)
        return None

def list_tables():
    """List all available tables in the database."""
    connection = create_connection()
    if not connection:
        print("Failed to connect to database")
        return None
    
    try:
        cursor = connection.cursor()
        cursor.execute("SHOW TABLES")
        tables = cursor.fetchall()
        
        if not tables:
            print("No tables found in the database")
            return None
        
        print("\nAvailable tables:")
        for i, table in enumerate(tables, 1):
            print(f"{i}. {table[0]}")
        
        return [table[0] for table in tables]
    
    except Error as e:
        print(f"Error listing tables: {e}")
        return None
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("Database connection closed")

def download_table(table_name, output_format='csv'):
    """Download an entire table from MariaDB.
    
    Args:
        table_name: Name of the table to download
        output_format: Format of the output file ('csv' or 'pandas')
        
    Returns:
        DataFrame if output_format is 'pandas', otherwise None
    """
    connection = create_connection()
    if not connection:
        print("Failed to connect to database")
        return None
    
    try:
        cursor = connection.cursor(dictionary=True)
        
        # Check if table exists
        cursor.execute(f"SHOW TABLES LIKE '{table_name}'")
        if not cursor.fetchone():
            print(f"Table '{table_name}' does not exist in the database")
            return None
        
        # Get column names
        cursor.execute(f"DESCRIBE {table_name}")
        columns = [column['Field'] for column in cursor.fetchall()]
        
        # Get all data
        query = f"SELECT * FROM {table_name}"
        cursor.execute(query)
        rows = cursor.fetchall()
        
        if not rows:
            print(f"The table '{table_name}' is empty")
            return None
        
        # Count rows
        cursor.execute(f"SELECT COUNT(*) as count FROM {table_name}")
        row_count = cursor.fetchone()['count']
        print(f"Downloaded {row_count} rows from '{table_name}'")
        
        # Convert to DataFrame
        df = pd.DataFrame(rows)
        
        if output_format == 'csv':
            filename = f"{table_name}.csv"
            df.to_csv(filename, index=False)
            print(f"Data saved to {filename}")
            return None
        else:
            return df
    
    except Error as e:
        print(f"Error downloading table: {e}")
        return None
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("Database connection closed")

# Example usage - no need for command-line arguments
if __name__ == "__main__":
    # List all tables
    print("Listing all tables:")
    tables = list_tables()
    
    # Download the embeddings table
    print("\nDownloading 'embeddings' table:")
    download_table("embeddings")

Listing all tables:
Successfully connected to MariaDB at 86hpm.h.filess.io
Connected to server version 5.5.5-10.7.4-MariaDB-1:10.7.4+maria~focal
Database: properties_forgetvery

Available tables:
1. embeddings
Database connection closed

Successfully connected to MariaDB at 86hpm.h.filess.io
Connected to server version 5.5.5-10.7.4-MariaDB-1:10.7.4+maria~focal
Database: properties_forgetvery
Downloaded 99 rows from 'embeddings'
Data saved to embeddings.csv
Database connection closed
